In [ ]:
import re,os
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(data, key=alphanum_key)

print(sorted_alphanumeric(os.listdir('./04-11-2023/PT')))

['PT_0_to_99.txt', 'PT_100_to_199.txt', 'PT_200_to_299.txt', 'PT_300_to_399.txt', 'PT_400_to_499.txt', 'PT_500_to_599.txt', 'PT_600_to_699.txt', 'PT_700_to_799.txt', 'PT_800_to_899.txt', 'PT_900_to_999.txt', 'PT_1000_to_1099.txt', 'PT_1100_to_1199.txt', 'PT_1200_to_1299.txt', 'PT_1300_to_1399.txt', 'PT_1400_to_1499.txt', 'PT_1500_to_1599.txt', 'PT_1600_to_1699.txt', 'PT_1700_to_1799.txt', 'PT_1800_to_1899.txt', 'PT_1900_to_1999.txt', 'PT_2000_to_2099.txt', 'PT_2100_to_2199.txt', 'PT_2200_to_2299.txt', 'PT_2300_to_2399.txt', 'PT_2400_to_2499.txt', 'PT_2500_to_2599.txt', 'PT_2600_to_2699.txt', 'PT_2700_to_2799.txt', 'PT_2800_to_2899.txt', 'PT_2900_to_2999.txt', 'PT_3000_to_3099.txt', 'PT_3100_to_3199.txt', 'PT_3200_to_3299.txt', 'PT_3300_to_3399.txt', 'PT_3400_to_3499.txt', 'PT_3500_to_3599.txt', 'PT_3600_to_3699.txt', 'PT_3700_to_3799.txt', 'PT_3800_to_3899.txt', 'PT_3900_to_3999.txt', 'PT_4000_to_4099.txt', 'PT_4100_to_4199.txt', 'PT_4200_to_4299.txt', 'PT_4300_to_4399.txt', 'PT_4400_t

In [ ]:
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

time: 0 ns (started: 2024-04-12 15:21:02 +05:30)


In [ ]:
from tqdm.notebook import tnrange
from scipy.stats import pearsonr
import pandas as pd
import numpy as np
from IPython.display import clear_output
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
import os
import re

In [1]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(data, key=alphanum_key)

def batch_hex_txt_to_npy(dir_path,array_name):
#     filename = r'./1k-01/New folder/textin_array.txt'
    hex_string_list=[]
    file_list = sorted_alphanumeric(os.listdir(dir_path))
    for file_index in tnrange(len(os.listdir(dir_path)),desc="Processing PT/CT files"):
        if(file_list[file_index].endswith(".txt")):
            filepath = os.path.join(dir_path,file_list[file_index])
            with open (filepath,'r') as fp:
                file_data_list = fp.readlines()
            for item in file_data_list:
                hex_string_list.append(item)
    for item in range(len(hex_string_list)):
        hex_string_list[item]=hex_string_list[item].split('\n')[0]
        byte_array=[]
        for pos in range(0,len(hex_string_list[item]),2):
            byte_array.append(int(hex_string_list[item][pos:pos+2],16))
        hex_string_list[item] = byte_array
    npy_array=np.array(hex_string_list)
    np.save(f'./traces/{array_name}', npy_array)

def int_trace_to_npy(dir_path,array_name):
    #dir_path = input(r'Input the path of the folder:')
    trace_list = []
    file_list = sorted_alphanumeric(os.listdir(dir_path))
    for filenum in tnrange(len(os.listdir(dir_path)),desc="Processing File"):
        if(file_list[filenum].endswith(".txt")):
            filepath = os.path.join(dir_path,file_list[filenum])
            with open(filepath,'r')as fp:
                wavedesc = fp.read()
            vertical_gain=0
            vertical_offset=0
            arr=[]
            for header in wavedesc[1:].split('"\n')[0].split("\n"):
                if header.startswith("VERTICAL_GAIN"):
                    vertical_gain = float(header.split(':')[-1].strip())
                if header.startswith("VERTICAL_OFFSET"):
                    vertical_offset = float(header.split(':')[-1].strip())

            for val in wavedesc[1:].split('"\n')[-1].split("\n")[:-1]:
                arr.append(round(((float(val))*vertical_gain)-vertical_offset,5))
#                 arr.append(val)
            trace_list.append(arr)
    np.save(f'./traces/{array_name}',np.array(trace_list))

# def txt_trace_to_npy(dir_path,array_name):
#     #dir_path = input(r'Input the path of the folder:')
#     trace_list = []
#     file_list = sorted_alphanumeric(os.listdir(dir_path))
#     for filenum in tnrange(len(os.listdir(dir_path)),desc="Processing trace files"):
#         if(file_list[filenum].endswith(".txt")):
#             filepath = os.path.join(dir_path,file_list[filenum])
#             arr = np.loadtxt(filepath, delimiter='\n', max_rows=12500)
# #             arr=arr-arr.mean()
# #             arr=arr/arr.max()
#             if(len(arr)!=12500):
#                 print(f"{file} does not have 12500 points")
#             trace_list.append(arr)
#     np.save(f'./traces/traces_80k/{array_name}',np.array(trace_list))

# def csv_to_npy(dir_path,array_name):
# #     dir_path = r'C:\Users\pschannels\Desktop\Vinod\Analysis\1k-01'
#     file_list = []
#     for filename in sorted_alphanumeric(os.listdir(dir_path)):
#         if(filename.endswith(".csv")):
#             filepath = os.path.join(dir_path,filename)
#             arr = np.loadtxt(filepath, delimiter='\t', skiprows = 3, max_rows=12500)
#             file_list.append(arr)
# #         if(len(arr)!=500):
# #             print(f"{filename} does not have 12500 points")
#     np.save(f'./traces/traces_80k/{array_name}',np.array(file_list))

In [ ]:
batch_hex_txt_to_npy(r'./04-11-2023/PT','textin_array_04_11_2023')
batch_hex_txt_to_npy('./04-11-2023/CT','textout_array_04_11_2023')
int_trace_to_npy('./04-11-2023/','trace_array_04_11_2023')
# batch_hex_txt_to_npy(r'./27-10-2022/PT','textin_array_09_10_2023')
# batch_hex_txt_to_npy('./09-10-2023/CT','textout_array_09_10_2023')
# int_trace_to_npy('./09-10-2023/','trace_array_09_10_2023')

Processing PT/CT files:   0%|          | 0/300 [00:00<?, ?it/s]

Processing PT/CT files:   0%|          | 0/300 [00:00<?, ?it/s]

Processing File:   0%|          | 0/30002 [00:00<?, ?it/s]

time: 8min 8s (started: 2023-11-07 11:44:23 +05:30)


In [ ]:
    textin_array = np.load('./traces/textin_array_04_11_2023.npy')
textout_array = np.load('./traces/textout_array_04_11_2023.npy')
# trace_array = np.load('./traces/trace_array.npy')
trace_array = np.load('./traces/trace_array_04_11_2023.npy')
# trace_array_old = np.load('./traces/trace_array_27_10_2022.npy')

# print(known_keys.shape)
print(textin_array.shape)
print(textout_array.shape)
print(trace_array.shape)

(30000, 16)
(30000, 16)
(30000, 12502)
time: 19.4 s (started: 2024-04-12 15:21:25 +05:30)


In [ ]:
output_notebook()
p = figure(plot_width = 1000, plot_height = 400)
p.line(range(len(trace_array[1])), trace_array[1])
show(p)

Loading BokehJS ...

time: 47 ms (started: 2024-04-12 15:22:25 +05:30)


In [ ]:
print(trace_array[0])

[0.99233 0.9923  0.99252 ... 0.99248 0.99251 0.99245]
time: 0 ns (started: 2023-11-08 12:15:17 +05:30)


In [ ]:
def dec_to_hex(arr):
    s = "".join([str(hex(x)[2:]).rjust(2, "0") for x in arr])
    return s

def hex_to_dec(s):
    deci_list = []
    for i in range(0, len(s), 2):
        deci_list.append(int(s[i:i+2], 16))
    return deci_list

time: 0 ns (started: 2023-11-08 12:15:17 +05:30)


In [ ]:
textin_array = np.array(textin_array, dtype = np.int32)
textout_array = np.array(textout_array, dtype = np.int32)

time: 0 ns (started: 2023-11-08 12:15:18 +05:30)


In [ ]:
print(dec_to_hex(textin_array[0]))
print(textin_array[0])
print(hex_to_dec(dec_to_hex(textin_array[0])))
print(dec_to_hex(textout_array[0]))

fa0221e537d287ce2b377de59a49e8fe
[250   2  33 229  55 210 135 206  43  55 125 229 154  73 232 254]
[250, 2, 33, 229, 55, 210, 135, 206, 43, 55, 125, 229, 154, 73, 232, 254]
8da01c59b6f8daab5f6c390a401c8f64
time: 16 ms (started: 2023-11-08 12:15:19 +05:30)


In [ ]:
def sbox(inp):
    s =  [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67,
            0x2b, 0xfe, 0xd7, 0xab, 0x76, 0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59,
            0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0, 0xb7,
            0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1,
            0x71, 0xd8, 0x31, 0x15, 0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05,
            0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75, 0x09, 0x83,
            0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29,
            0xe3, 0x2f, 0x84, 0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b,
            0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf, 0xd0, 0xef, 0xaa,
            0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c,
            0x9f, 0xa8, 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc,
            0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2, 0xcd, 0x0c, 0x13, 0xec,
            0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19,
            0x73, 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee,
            0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb, 0xe0, 0x32, 0x3a, 0x0a, 0x49,
            0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
            0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4,
            0xea, 0x65, 0x7a, 0xae, 0x08, 0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6,
            0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, 0x70,
            0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9,
            0x86, 0xc1, 0x1d, 0x9e, 0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e,
            0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf, 0x8c, 0xa1,
            0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0,
            0x54, 0xbb, 0x16]
    return s[inp]

def invsbox(inp):
    s = [0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3,
            0x9e, 0x81, 0xf3, 0xd7, 0xfb , 0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f,
            0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb , 0x54,
            0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b,
            0x42, 0xfa, 0xc3, 0x4e , 0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24,
            0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25 , 0x72, 0xf8,
            0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d,
            0x65, 0xb6, 0x92 , 0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda,
            0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84 , 0x90, 0xd8, 0xab,
            0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3,
            0x45, 0x06 , 0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1,
            0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b , 0x3a, 0x91, 0x11, 0x41,
            0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6,
            0x73 , 0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9,
            0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e , 0x47, 0xf1, 0x1a, 0x71, 0x1d,
            0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b ,
            0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0,
            0xfe, 0x78, 0xcd, 0x5a, 0xf4 , 0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07,
            0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f , 0x60,
            0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f,
            0x93, 0xc9, 0x9c, 0xef , 0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5,
            0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61 , 0x17, 0x2b,
            0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55,
            0x21, 0x0c, 0x7d]
    return s[inp]



INVSHIFT_undo = [0, 5, 10, 15, 4, 9, 14, 3, 8, 13, 2, 7, 12, 1, 6, 11]

def invshiftRows(pt, keyguess, bnum):
    return pt[bnum] ^ keyguess

def intermediate(pt, keyguess, bnum, HD):
    if HD:
        return invsbox(invshiftRows(pt, keyguess, bnum)) ^ pt[INVSHIFT_undo[bnum]]
    else:
        return invsbox(invshiftRows(pt, keyguess, bnum))

# def leakage_1(ct, key, bnum):
#     st9 = invsbox(ct[bnum] ^ key)
#     # return (st9 ^ st10)
#     return st9

# def leakage_2(ct, key, bnum):
#     st10 = ct[INVSHIFT_undo[bnum]]
#     st9 = invsbox(ct[bnum] ^ key)
#     return (st9 ^ st10)

# def intermediate(ct, key, bnum):
#     return leakage_2(ct, key, bnum)

HW = [bin(n).count("1") for n in range(0, 256)]

In [ ]:
fmt = "{:02X}<br>{:.3f}<br>{:}"
def format_stat(stat):
    return str(fmt.format(stat[0], stat[1], stat[2]))

def color_corr_key(row):
    ret = [""] * len(row)
    for i, bnum in enumerate(row):
        if i == 0:
            ret[i] = "color: green"
        else:
            ret[i] = "color: red"
    return ret

In [ ]:
def getroundkey(startTime, endTime, text_array, numtraces, corr_rk):
    roundkey_guess = []

    totaltraces = trace_array.shape[0]
    # numtraces = 20_000
    numpoints = trace_array.shape[1]

    start_point = startTime
    end_point = endTime

    bigR_matrix = np.zeros((16, 256, numpoints))

    H_matrix = np.zeros((numtraces, 256), dtype = np.float32)

    # for printing the top guessed keys
    printable = []

    for subkey in tnrange(0, 16, desc="Attacking Subkey"):
        temp = []
        for kguess in tnrange(0, 256, desc="Generating Hamming Weights"):
            for trace_no in range(numtraces):
                iv = intermediate(text_array[trace_no], kguess, subkey, True)
                H_matrix[trace_no, kguess] = HW[iv]

            for point in range(start_point, end_point):
                hw = H_matrix[:numtraces, kguess]
                trc = trace_array[:numtraces, point]
                if (np.var(hw, ddof = 1) == 0) or (np.var(trc, ddof = 1) == 0):
                    if (np.var(hw, ddof = 1) == 0) and (np.var(trc, ddof = 1) == 0):
                        print("Hi")
                    elif (np.var(hw, ddof = 1) == 0):
                        print("HW")
                    else:
                        print("TRC")
                    bigR_matrix[subkey, kguess, point] = 0
                else:
                    bigR_matrix[subkey, kguess, point] = np.abs(pearsonr(hw, trc)[0])

            temp.append((kguess, np.max(bigR_matrix[subkey, kguess]), np.argmax(bigR_matrix[subkey, kguess])))

        temp[0], temp[corr_rk[subkey]] = temp[corr_rk[subkey]], temp[0]
        temp1 = temp[1:]
        temp1.sort(key = lambda x: -x[1])
        temp[1:] = temp1
        printable.append(temp)
        df = pd.DataFrame(printable).transpose()

        roundkey_guess.append(bigR_matrix[subkey].max(axis = 1).argmax())

        # clears previous output
        clear_output(wait=True)
        display(df.head().style.format(format_stat).apply(color_corr_key, axis = 0))

    return roundkey_guess, bigR_matrix

time: 16 ms (started: 2023-11-08 12:15:20 +05:30)


In [ ]:
roundkey10_guess = getroundkey(startTime = 10_000, endTime = 10_500, text_array = textout_array, numtraces = 0_000, corr_rk = hex_to_dec('13111d7fe3944a17f307a78b4d2b30c5'))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,130.07710131,110.08410152,1D0.07910129,7F0.09310131,E30.07410131,940.06810154,4A0.07610132,170.07910153,F30.07310154,070.08510129,A70.06510128,8B0.08710133,4D0.11910132,2B0.07710132,300.07610129,C50.08410131
1,8B0.03710128,7D0.02610019,700.02710004,6A0.02710149,260.03710132,9D0.02710100,D60.02510003,520.02410431,F10.04810131,B50.02510414,360.02610277,610.02610380,B20.04510132,E70.02610111,150.02810088,C80.02510488
2,450.03310134,D20.02510138,2A0.02610487,A30.02710237,5D0.03610128,2E0.02610305,A30.02510408,390.02310075,500.04210155,680.02510072,120.02410340,A40.02510346,CF0.04410133,6B0.02510272,AD0.02610007,ED0.02510116
3,9A0.03310108,0C0.02510297,B70.02610132,520.02610370,D60.03410135,6C0.02610000,210.02310149,A30.02310295,DB0.03710134,7D0.02410374,180.02410108,E10.02410035,DF0.04110129,3B0.02410252,F00.02510081,3F0.02310353
4,550.03210129,450.02410316,170.02510186,C60.02510496,4A0.03310134,6D0.02510085,C50.02310345,2A0.02310460,010.03210152,170.02410484,5A0.02310149,900.02410370,1A0.04010132,430.02310059,D50.02310419,D40.02310026


time: 2h 4min 44s (started: 2023-11-08 12:15:22 +05:30)


In [ ]:
%unload_ext autotime